In [6]:
from transformers import BertTokenizerFast, BertModel
import torch

print(torch.cuda.is_available())

True


In [7]:
class SummarizationClassifier(torch.nn.Module):
    def __init__(self):
        super(SummarizationClassifier, self).__init__()
        self.base_model = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = torch.nn.Dropout(0.1)
        self.linear = torch.nn.Linear(768, 2)
        self.sigmoid = torch.nn.Sigmoid()
        
        if torch.cuda.is_available():
            self.device = torch.device("cuda")
        else:
            self.device = torch.device("cpu")
        self.to(self.device)
        
    def forward(self, inputs):
        outputs = self.base_model(**inputs)
        outputs = self.dropout(outputs[1])
        outputs = self.linear(outputs)
        
        return self.sigmoid(outputs)

In [8]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
model = SummarizationClassifier()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
f = open("data\\data_1.txt", encoding="utf8")
data = f.read()
f.close()
inputs = tokenizer(data, max_length=512, truncation='longest_first', return_tensors="pt")
inputs.to(model.device)
outputs = model(inputs)

In [10]:
print(outputs)
torch.nn.functional.softmax(outputs, dim=1)

tensor([[0.5604, 0.5640]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[0.4991, 0.5009]], device='cuda:0', grad_fn=<SoftmaxBackward0>)